In [1]:
pip install https://github.com/buguroo/pyknow/archive/v1.7.0.tar.gz

     / 1.4 MB 5.1 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34137 sha256=837bf8999e9e4f58071806400e119c787ae58dd6b2d8039b85526fea461f296a
  Stored in directory: /tmp/pip-ephem-wheel-cache-8mn3ix11/wheels/31/ad/cd/8761b426dfcdeed085eac4b53cfaba4c4b351632717142def9
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3149 sha256=fc71156e9284912c1d8cfa9a011bb4d8553cd60fdf52a0c6dbb9ea8461ba09e9
  Stored in directory: /root/.cache/pip/wheels/ff/80/86/2d516a3c26397f67adaa2c848879d4a6359d90a60546ce4e03
Successfully built pyknow frozendict
  Attempting uninstall: frozendict
    Found existing installation: frozendict 2.4.2
    Uninstalling frozendict-2.4.2:
      Successfully uninstalled frozendict-2.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the f

In [2]:
!pip install --upgrade frozendict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: frozendict
    Found existing installation: frozendict 1.2
    Uninstalling frozendict-1.2:
      Successfully uninstalled frozendict-1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyknow 1.7.0 requires frozendict==1.2, but you have frozendict 2.4.2 which is incompatible.


In [22]:
from pyknow import *

# Define the Fact structure for user input and system state
class LandlordDecision(Fact):
    # These are the fields that will hold the facts for the expert system
    Case_Status = Field(bool, mandatory=False)  # Lease renewal status
    Continue_Filing = Field(bool, mandatory=False)  # Acceptance of rent increase
    Mediation = Field(bool, mandatory=False)  # Not used in the current rules
    Order = Field(bool,mandatory=False)  # Not used in the current rules
    Challenge = Field(bool,mandatory=False)  # Not used in the current rules
    Explanation_Status = Field(bool,mandatory=False)  # Agreement status on lease renewal

# Expert system knowledge engine for contacting LTB
class File_a_case_LTB(KnowledgeEngine):
    # Initial user query
    @DefFacts()
    def startup(self):
        # Ask the tenant whether he wants to renew the lease
        yield LandlordDecision(Case_Status=input("Do you want to file a case with LTB? (yes/no)") == 'yes')
    @Rule(LandlordDecision(Case_Status=False))
    def leaving(self):
        print("Thank you for contacting LTB! More support coming soon!")
        self.halt()

    # If the tenant wants to renew the lease, ask about the rent increase
    @Rule(LandlordDecision(Case_Status=True))
    def extending(self):
        print("""
If you have a problem with your landlord tenant, you should talk to each other first. Problems can often be solved with a conversation.
If you can’t solve the problem by talking about it, there are other steps to take before applying to the LTB.
For example:
If you are a landlord and your tenant isn’t paying rent, you usually need to give your tenant an eviction notice before you file.
These and other steps are covered in detail in the next section, Choosing an Application.

You might also want to use our online tool, Navigate Tribunals Ontario, to find out about your rights and responsibilities and what application you can file. You can also read our Renting in Ontario webpage or get legal advice before you apply.
        """)
        self.declare(LandlordDecision(Case_Status=True,Continue_Filing=input("Do you still wish to proceed with filing the application? (yes/no)")=='yes'))
    @Rule(LandlordDecision(Case_Status=False,Continue_Filing=False))
    def not_extending(self):
        print("Thank you for contacting LTB! Hope you already know how to file the application!")
        self.halt()

    # If the tenant is okay with the rent increase, ask if the lease is renewed
    @Rule(LandlordDecision(Case_Status=True, Continue_Filing = True))
    def approval_extending(self):
        print("""
The application you choose to file will depend on what you want to happen. If you are not sure which application to file, use our online tool, Navigate Tribunals Ontario.

If you are a non-profit housing co-op or a co-op member, visit Non-Profit Co-op Evictions
        """)
        self.declare(LandlordDecision(Continue_Filing = True,Case_Status=True,Explanation_Status=input("Do you want to know how to file the application (Steps to file the application)? yes/no")=='yes'))
    # If the tenant does not agree to the rent increase, end the renewal process
    @Rule(LandlordDecision(Case_Status=True,Continue_Filing = True,Explanation_Status = False))
    def end_renewal(self):
        print("Thank you for contacting LTB, hope you already know how to file the application!")
        self.halt()

    # If the lease is renewed, print a success message
    @Rule(LandlordDecision(Case_Status=True,Continue_Filing = True, Explanation_Status =True))
    def agreement_reached(self):
        print("""
Get the application form
The LTB encourages the use of the Tribunals Ontario Portal for L1, L2 L3, L4, L9, L10, T1, T2, T5, and T6 applications. For most other LTB applications, you can file your application by email and pay online. You can also download the LTB forms or contact us to have the forms mailed to you.

Complete the application form
Read the instructions carefully. The instructions will tell you if there is anything you need to do before you file. The instructions explain how to complete the application, what to do when the application is completed, and whether there are any documents you need to file with the application. If you don’t file all the documents you are supposed to or you make a mistake on your form, your application may be dismissed.

Requesting an Accommodation
If you require an accommodation to participate in a hearing, or you believe the format will result in an unfair hearing (for example: you do not have access to a telephone, computer and/or the internet), please refer to the Request for Accommodation form.

You can submit this form to the LTB by email to LTB@ontario.ca, or by mail or courier to 15 Grosvenor Street, Ground Floor, Toronto, ON M7A 2G6. Be sure to include your file number.

Please send this form to the LTB as soon as possible so that the LTB has enough time to consider your request before the hearing.

For more information, refer to Tribunals Ontario’s Accessibility and Accommodation Policy and Practice Direction on Hearing Formats.

If you are concerned about sharing sensitive medical information with the LTB, contact the LTB.

File an Application
The LTB encourages the use of the Tribunals Ontario Portal for the following applications:

Landlord Applications

L1: Application to evict a tenant for non-payment of rent and to collect rent the tenant owes
L2: Application to End a Tenancy and Evict a Tenant or Collect Money
L3: Application to End a Tenancy – Tenant Gave Notice or Agreed to Terminate the Tenancy
L4: Application to End a Tenancy and Evict a Tenant – Tenant Failed to Meet Conditions of a Settlement or Order
L9 : Application to Collect Rent the Tenant Owes
L10: Application to Collect Money a Former Tenant Owes

For most other LTB applications, you can file your application by email and pay online.

Applications can also be sent by mail or courier to your LTB regional office.

Some ServiceOntario Centres accept all Landlord and Tenant Board applications in-person. Please visit ServiceOntario Centres for a list of locations that accept applications and documents on behalf of the LTB.

You are responsible for making sure your form is complete and correct. The LTB will review your application. In some circumstances, applications are screened for errors and will be returned to the client if necessary.

If you are responding to a case on the Tribunals Ontario Portal, you must have the file number and the unique one-time pin from the LTB. The file number and unique pins are emailed and mailed to responding parties who have a file on the Tribunals Ontario Portal.

The LTB schedules a hearing
Once your application is accepted, the LTB will send you a Notice of Hearing which will include the following:

the type of application that has been filed
the purpose of the hearing
the type of hearing that will be held (videoconference or phone)
the date, time and type of hearing
In most cases, the LTB will mail you and the other parties a copy of the application and the Notice of Hearing but the LTB could order you to deliver (“serve”) these documents on the other parties. If you need to serve the documents, the LTB will give you an “Order to Serve Documents” that will tell you what documents you must serve and when you must serve them. There are strict rules about how to serve these documents. For more information, see: How to Serve an Application and Notice of Hearing and Rule 10 – Service of Application and Notice of Hearing.

The LTB will only serve a copy of the application and the Notice of Hearing. The LTB will not serve any documents or photos that you want to use as evidence to support your application. All parties must provide the other parties with a complete copy of the evidence they intend to rely upon at least 7 days before the hearing or 5 days in the case of responding evidence, unless the LTB orders or directs otherwise.

Evidence should be submitted using the Tribunals Ontario Portal. Read the Practice Direction on Evidence for more information about how and when to submit evidence if you have an upcoming hearing. You must also serve the other parties with a copy of all the evidence you uploaded into the portal unless the other parties have consented in writing to accept service through the portal. The parties can use the Consent to Disclosure through Tribunals Ontario Portal form for such an agreement.

At the hearing, the LTB member (adjudicator) will decide what documents will be accepted as evidence.

Timelines for next steps
On average, L1 and L9 applications are being scheduled within five months. A Notice of Hearing will be sent to the parties before the scheduled hearing date. This is an improvement from earlier this year, when the average was 8-10 months to get to a hearing.

On average, other new and adjourned matters are currently being scheduled within seven to eight months from when they were received or adjourned.


        """)
        self.declare(LandlordDecision(Continue_Filing = True,Case_Status=True,Explanation_Status=True,Mediation = input("Do you want to know about the Mediation and the Hearning Informations? yes/no")=='yes'))

    # If the lease is not renewed, inform the tenant of their rights
    @Rule(LandlordDecision(Continue_Filing = True,Case_Status=True,Explanation_Status=True,Mediation =False))
    def mediationfunn(self):
        print("Thank you for contacting LTB, hope you already know informations about Mediation and hearing!")
        self.halt()
    @Rule(LandlordDecision(Continue_Filing = True,Case_Status=True,Explanation_Status=True,Mediation = True))
    def Mediationfun(self):
        print("""
Preparing for your hearing day
You might want to get legal advice before your hearing.

The LTB has rules and practice directions that apply to its processes, and interpretation guidelines that explain how the LTB might decide specific issues in an application. You can read them in Rules, Practice Directions and Guidelines. The Practice Direction on Evidence contains information about how and when to submit evidence if you have an upcoming hearing.

You might also want to read decisions of the LTB.

Mediation
During mediation, a neutral person talks with you and the other party to see if both sides can come up with a solution to the problem.

Parties using Tribunals Ontario Portal can use the Online Dispute Resolution (ODR) tool to negotiate, among themselves, a settlement of the issues in dispute.

If the parties reach an agreement, they can ask the LTB for assistance and a Dispute Resolution Officer (DRO) will be assigned to the matter. This is also the case if the parties reach an impasse. A DRO can help the parties determine if they wish to engage with mediation as the next step in the process, or if the matter will proceed to a hearing instead.

ODR is not mediation, and it’s not adjudication. It’s the first step in determining which of these two avenues is most appropriate based on the position of the parties. It is a negotiation between parties in a safe space which is monitored for abuse and which provides tips on negotiation. It’s also the space where people can request the assistance of a DRO.

ODR can result in early resolution of an application through a payment plan, consent order or mediated agreement with the assistance of a Dispute Resolution Officer.

ODR may also result in the applicant wishing to withdraw their application, in which case a withdrawal request can be submitted through the portal.

Some other benefits of ODR include easier access to a DRO, having all file information in one easily accessible place, and accessing documents/evidence quickly and easily.

Types of hearings
There are two types of hearings at the LTB. The LTB decides what type of hearing to hold.

During a telephone or video hearing, the parties file any documents or materials before the hearing. During the hearing, they give evidence and explain their position by telephone or by video. The LTB has information available for participating in a Zoom hearing and for participating in a telephone hearing.
For a written hearing, the parties file a written explanation of their position and supporting documents. The member considers the explanation and reviews the documents filed to make their decision. Written hearings are most commonly used to decide Applications for an Above Guideline Increase and Applications to Vary the Amount of a Rent Reduction.
Who goes to the hearing
Applicant(s): The person who filed the application (i.e. the landlord or the tenant) or a person who is appearing in their place.

Respondent(s): The other person named in the application (i.e. the landlord or the tenant) or a person who is appearing in their place.

The applicant and the respondent are called the parties to the application.

Witnesses: Witnesses are people who can give evidence (information) that is relevant to the dispute. Witnesses will usually be asked to give information rather than offer their opinion. For example, a witness may describe something they saw or heard.

You need to make sure that your witnesses participate at the right time, day and videoconference. If you are worried that a witness might not show up, you can complete a Request for Summons. A member will review the request and decide if a summons should be issued. If the member thinks the person will not be able to provide relevant evidence, the request will be refused.

If a witness needs to take time off work to go to the hearing, they might ask you for a summons to show to their employer.

If a witness is summoned to a hearing, you will need to personally deliver (“serve”) the summons to them and pay them witness fees. For more information, read the brochure Requesting a Summons.

Representatives: You do not need to have a legal representative, but if you do, the representative will also attend the hearing. Your representative will speak on your behalf at the hearing, question witnesses, submit evidence and make written submissions.

A representative may be a lawyer or paralegal you hire to act on your behalf. You may also choose a person who is not a lawyer or paralegal to represent you. This could be a friend or relative or someone who is allowed by the Law Society to provide legal services without a licence. If you cannot attend your hearing and your representative is not a lawyer or paralegal, you will need to give your representative permission in writing to act on your behalf. For more information, see the Practice Direction on Representation before the LTB.

If you don’t go the hearing and no one attends in your place, the member can dismiss the application or decide it without you.

Support Persons: A support person, such as a family member or friend, can attend a hearing or mediation with you. A support person is not considered a representative as long as they don’t speak for you in the hearing or mediation.

Litigation Guardians: Sometimes an application will involve a party without “legal capacity” either because the person does not have the mental capacity to make decisions about the issues in the case or, in certain cases, because the person is under 18. That person can participate in the hearing through a litigation guardian. For more information, see the Practice Direction on Litigation Guardians.

What happens during the hearing:
The notice of hearing gives you the date and time of your hearing. A number of files will have the same start time in what is called a “hearing block”.

Be sure to log onto your video link or call at least 15 minutes prior to your hearing. If you do not sign-in and you are not in the hearing room when your case is called, your hearing may go ahead without you.

Wait in the virtual video conference hearing room unless the member has said you can leave and come back at a certain time.

You may have to wait for much of the day, so make arrangements for child care or to be absent from work or school. If it’s not possible to finish your hearing that day you’ll be given a new hearing date.

During the hearing, you and the other party will have a chance to question witnesses, introduce relevant documents as evidence and make arguments about the facts and the law.

The member controls the hearing. Because the member is neutral, he or she cannot provide legal advice or tell you how to present your case. It is up to you to present evidence that supports your position. The member may ask questions during the hearing.

Everyone at the hearing is expected to be courteous and respectful. See Rules of Procedure A7.

When the hearing is over, the member might tell you their decision right away or they might “reserve” the decision, which means they will take more time to consider your evidence and submissions. In either case, you will receive the decision in writing explaining the result. This decision is called an order.

Resolving Matters Without a Hearing
In some cases, an application may be resolved without a hearing. Parties can resolve matters that are not complex using the forms below to help the LTB understand if there have been changes after an application has been submitted. The applicant can use the Request for a Discontinuance Order Without a Hearing Form and/or a Request to Withdrawal an Application form to tell the LTB that:

The applicant wants to withdraw the application
An order can be issued because the application is discontinued; the tenant has paid in full all arrears and the application fee
An order can be issued because the parties have agreed to a payment agreement
Applicants can submit the form using the Tribunals Ontario Portal. If you are unable to file on the portal, you can email the form to LTB@Ontario.ca. Please include your file number and the title of your request in the subject line.
        """)
        self.declare(LandlordDecision(Continue_Filing = True,Case_Status=True,Explanation_Status=True,Mediation = True, Order =  input("Do you want to know more about the Orders that LTB issues? yes/no")=='yes'))
    # If the lease is not renewed, inform the tenant of their rights
    @Rule(LandlordDecision(Continue_Filing = True,Case_Status=True,Explanation_Status=True,Mediation =True, Order = False))
    def ordernn(self):
        print("Thank you for contacting LTB, hope you already know informations about Orders that LTB issues!")
        self.halt()
    @Rule(LandlordDecision(Continue_Filing = True,Case_Status=True,Explanation_Status=True,Mediation = True, Order = True))
    def ordern(self):
        print("""
The member who hears the application (or reviews the documents filed, in the case of a written hearing) will make the final order.

The order tells the parties, in writing, what the member has decided and may describe terms or conditions that a party must follow. For example, the member could order that the tenancy is terminated, or that repairs be made to the rental unit.

Usually, the LTB will mail a copy of the order to all of the parties named in the application and their legal representative.

You can visit Navigate Tribunals Ontario to learn more about what you can do after receiving an order from the LTB.

About reasons
An order may include written reasons. The reasons explain how the member analysed the evidence to arrive at their decision.

You can ask for reasons to be included in the order:

while you are at the hearing, or
in writing, within 30 days of the date on the order.
Types of orders
There are four common orders:

an interim order
an ex parte order
a hearing order
a consent order
Interim order
An interim order is an order that tells a party to do something, or decides part of the application, before the final order is issued.

Ex-parte order
An ex parte order is made when the member reviews the documents filed by the applicant and makes a decision, without notifying the respondent. Only two application types can be decided ex parte:

An application filed by the landlord to end the tenancy because the tenant agreed to move out on a specific date or gave notice to move out on a specific date (L3 application)
An application filed by the landlord to end the tenancy because the tenant did not meet the terms of a mediated agreement or order (L4 application).
Hearing order
A member issues a hearing order after a hearing.

Consent order
The LTB can issue a consent order when:

a tenant and a landlord have settled their dispute through an agreement and ask for the terms of the settlement to be put into a consent order. The terms of a consent order (e.g. a tenant agrees to move out of the apartment on a specific date) can be enforced by the court.
a landlord files an application to evict a tenant for non-payment of rent (Form L1) or for arrears of rent (Form L9) if:
the landlord and tenant work out a payment plan, and
a copy of the written payment plan is filed with the LTB before the hearing.
In this case, no hearing is held and the consent order is based on the payment plan that landlord and the tenant have agreed to.

Voiding an LTB order
An eviction order issued by the LTB for non-payment of rent usually allows the tenant to void the order by paying all the money they owe by the deadline in the order.

If the tenant pays the landlord the money they owe before the deadline, the tenant can file a Tenant's Affidavit and Motion to Void an Eviction Order for Arrears of Rent. A member will decide if the eviction order is void without holding a hearing.

If the tenant pays all the money owed after the deadline, the tenant can file a Tenant's Affidavit and Motion to Void an Eviction Order for Arrears of Rent. In this case, a member will hold a hearing to decide if the eviction order is void.

Enforcement of an LTB order
Most of the terms and conditions of LTB orders can be enforced through the courts. For example, if the tenant does not move out of the rental unit by the date in the eviction order, the order can be filed with the Court Enforcement Office (also known as the Sheriff’s office) to be enforced. An order for payment of money can be filed with the Small Claims Court for enforcement.

Timelines for receiving an order
As of June 2023, on average, orders are being sent within 60 days of a hearing.
        """)
        self.declare(LandlordDecision(Continue_Filing = True,Case_Status=True,Explanation_Status=True,Mediation = True, Order = True, Challenge =  input("Do you want to know more about Challenging or Correcting the order? yes/no")=='yes'))
    # If the lease is not renewed, inform the tenant of their rights
    @Rule(LandlordDecision(Continue_Filing = True,Case_Status=True,Explanation_Status=True,Mediation = True, Order = True, Challenge =False))
    def challengenn(self):
        print("Thank you for contacting LTB, hope you already know informations about Challenging and correcting the orders!")
        self.halt()
    @Rule(LandlordDecision(Continue_Filing = True,Case_Status=True,Explanation_Status=True,Mediation = True, Order = True, Challenge =True))
    def challengen(self):
        print("""
Once the LTB issues an order, it is final. The LTB will not change the order because a party does not like the decision or because a party believes that a different decision should have been made. For more information, read the brochure: Amendments, Reviews and Appeals.

Amending an order
If you notice a clerical mistake in the order, such as a spelling error in someone’s name or a calculation error, you have 30 days to file a Request to Amend an Order to ask the member to correct the mistake. In your request, tell the member what the mistake is and why it should be corrected. There is no fee to make this request. For details, see Interpretation Guideline 15: Amending an Order.

Asking the LTB to review an order
You can ask the LTB to review an order on the grounds that a “serious error” has been made in the order. A serious error can be:

LTB made an order it did not have the power to make
The application was not processed fairly and this interfered with the rights of one of the parties
The hearing was not conducted fairly and this interfered with the rights of one of the parties
There was no evidence to support the LTB’s conclusions
New evidence is available which was not available at the time of the hearing
A party was not reasonably able to participate in the LTB’s proceeding.
You need to file a Request to Review an Order within 30 days of the date of the order. There is a $58.00 fee.

You must explain in detail the error you believe the member made, and specify the changes you think should be made to the order.

If you want the order stayed (or put “on-hold’) until a decision can be made, you must ask for the stay, and give reasons why.

Review orders can:

deny the request for review
affirm the hearing order
order changes to the hearing order
For details, see Interpretation Guideline 8: Review of an Order.

Appealing an order
Any person affected by an LTB order can appeal the order to the Divisional Court. This court is the branch of the Superior Court of Justice that hears appeals from the decisions of boards and tribunals in Ontario.

An appeal of an order may only be made on a question of law. In general, this means a person must believe LTB’s order was not a reasonable interpretation or application of the law.

An appeal to Divisional Court must be filed within 30 days of the date the order was issued.

When an appeal of an order is filed with the Divisional Court, the order is automatically stayed, and cannot be enforced unless the Divisional Court or the LTB lifts (or removes) the stay at the request of a party.

The procedures for filing an appeal to the Divisional Court are set by the court, not by the LTB. You can contact the courthouse for more information about the appeal process. You can also download the Guide to Appeals in Divisional Court from the Divisional Court's website. You may want to get legal help.

Setting aside an ex parte order
The following applications may result in an ex parte order:

an application filed by the landlord to end the tenancy because the tenant agreed to move out on a specific date or gave notice to move out on a specific date (L3 application),
an application filed by the landlord to end the tenancy because the tenant did not meet the terms of a mediated agreement or order (L4 application).
If an ex parte order is issued, you can ask the LTB to set aside (or cancel) the order. A request to set aside the order is called a motion, which can be filed with the LTB by completing a Motion to Set Aside an Ex Parte Order. The motion must be filed with the LTB within 10 days from the date the order is issued. A hearing will be held and the member will make a decision.

When a motion to set aside an order is filed by the deadline, the order is automatically stayed (or put on hold) and it cannot be enforced until a final decision has been made.

For more information, see Motion to Set Aside an Ex Parte Order.

Re-opening an application
Only applications that were resolved by a consent order under section 206 of the RTA or an LTB mediated agreement can be re-opened.

There are two common situations when you can make the request:

The landlord or tenant did not meet one of the terms in the LTB mediated agreement
The landlord or tenant misled you or made you sign the mediated agreement
To ask to re-open an application, fill out the Request to Re-open an Application.
        """)
        print("Thanks for contacting LTB, Hope you had fun!")
        self.halt()
# Instantiate the knowledge engine
engine = File_a_case_LTB()
engine.reset()  # Reset the engine to its initial state
engine.run()  # Run the engine which will execute the rules based on the facts


Do you want to file a case with LTB? (yes/no)yes

If you have a problem with your landlord tenant, you should talk to each other first. Problems can often be solved with a conversation.
If you can’t solve the problem by talking about it, there are other steps to take before applying to the LTB. 
For example:
If you are a landlord and your tenant isn’t paying rent, you usually need to give your tenant an eviction notice before you file.
These and other steps are covered in detail in the next section, Choosing an Application.

You might also want to use our online tool, Navigate Tribunals Ontario, to find out about your rights and responsibilities and what application you can file. You can also read our Renting in Ontario webpage or get legal advice before you apply.
        
Do you still wish to proceed with filing the application? (yes/no)yes

The application you choose to file will depend on what you want to happen. If you are not sure which application to file, use our online tool, N

In [17]:
import streamlit as st

def main():
    st.title("Landlord Tenant Board Filing System")

    # Initial question
    file_case = st.radio("Do you want to file a case with LTB?", ('Yes', 'No'))

    if file_case == 'No':
        st.write("Thank you for contacting LTB! More support coming soon!")
    else:
        st.write("""
If you have a problem with your landlord tenant, you should talk to each other first. Problems can often be solved with a conversation.
If you can’t solve the problem by talking about it, there are other steps to take before applying to the LTB.
For example:
If you are a landlord and your tenant isn’t paying rent, you usually need to give your tenant an eviction notice before you file.
These and other steps are covered in detail in the next section, Choosing an Application.

You might also want to use our online tool, Navigate Tribunals Ontario, to find out about your rights and responsibilities and what application you can file. You can also read our Renting in Ontario webpage or get legal advice before you apply.
        """)

        proceed_with_filing = st.radio("Do you still wish to proceed with filing the application?", ('Yes', 'No'))
        if proceed_with_filing == 'No':
            st.write("Thank you for contacting LTB! Hope you already know how to file the application!")
        else:
             st.write("""
The application you choose to file will depend on what you want to happen. If you are not sure which application to file, use our online tool, Navigate Tribunals Ontario.

If you are a non-profit housing co-op or a co-op member, visit Non-Profit Co-op Evictions
        """)
          explanation_status = st.radio("Do you want to know how to file the application (Steps to file the application)?", ('Yes', 'No'))
          if explanation_status == 'No':
              st.write("""Thank you for contacting LTB, hope you already know how to file the application!""")
          else:
            st.write("""
  Get the application form
  The LTB encourages the use of the Tribunals Ontario Portal for L1, L2 L3, L4, L9, L10, T1, T2, T5, and T6 applications. For most other LTB applications, you can file your application by email and pay online. You can also download the LTB forms or contact us to have the forms mailed to you.

  Complete the application form
  Read the instructions carefully. The instructions will tell you if there is anything you need to do before you file. The instructions explain how to complete the application, what to do when the application is completed, and whether there are any documents you need to file with the application. If you don’t file all the documents you are supposed to or you make a mistake on your form, your application may be dismissed.

  Requesting an Accommodation
  If you require an accommodation to participate in a hearing, or you believe the format will result in an unfair hearing (for example: you do not have access to a telephone, computer and/or the internet), please refer to the Request for Accommodation form.

  You can submit this form to the LTB by email to LTB@ontario.ca, or by mail or courier to 15 Grosvenor Street, Ground Floor, Toronto, ON M7A 2G6. Be sure to include your file number.

  Please send this form to the LTB as soon as possible so that the LTB has enough time to consider your request before the hearing.

  For more information, refer to Tribunals Ontario’s Accessibility and Accommodation Policy and Practice Direction on Hearing Formats.

  If you are concerned about sharing sensitive medical information with the LTB, contact the LTB.

  File an Application
  The LTB encourages the use of the Tribunals Ontario Portal for the following applications:

  Landlord Applications

  L1: Application to evict a tenant for non-payment of rent and to collect rent the tenant owes
  L2: Application to End a Tenancy and Evict a Tenant or Collect Money
  L3: Application to End a Tenancy – Tenant Gave Notice or Agreed to Terminate the Tenancy
  L4: Application to End a Tenancy and Evict a Tenant – Tenant Failed to Meet Conditions of a Settlement or Order
  L9 : Application to Collect Rent the Tenant Owes
  L10: Application to Collect Money a Former Tenant Owes

  For most other LTB applications, you can file your application by email and pay online.

  Applications can also be sent by mail or courier to your LTB regional office.

  Some ServiceOntario Centres accept all Landlord and Tenant Board applications in-person. Please visit ServiceOntario Centres for a list of locations that accept applications and documents on behalf of the LTB.

  You are responsible for making sure your form is complete and correct. The LTB will review your application. In some circumstances, applications are screened for errors and will be returned to the client if necessary.

  If you are responding to a case on the Tribunals Ontario Portal, you must have the file number and the unique one-time pin from the LTB. The file number and unique pins are emailed and mailed to responding parties who have a file on the Tribunals Ontario Portal.

  The LTB schedules a hearing
  Once your application is accepted, the LTB will send you a Notice of Hearing which will include the following:

  the type of application that has been filed
  the purpose of the hearing
  the type of hearing that will be held (videoconference or phone)
  the date, time and type of hearing
  In most cases, the LTB will mail you and the other parties a copy of the application and the Notice of Hearing but the LTB could order you to deliver (“serve”) these documents on the other parties. If you need to serve the documents, the LTB will give you an “Order to Serve Documents” that will tell you what documents you must serve and when you must serve them. There are strict rules about how to serve these documents. For more information, see: How to Serve an Application and Notice of Hearing and Rule 10 – Service of Application and Notice of Hearing.

  The LTB will only serve a copy of the application and the Notice of Hearing. The LTB will not serve any documents or photos that you want to use as evidence to support your application. All parties must provide the other parties with a complete copy of the evidence they intend to rely upon at least 7 days before the hearing or 5 days in the case of responding evidence, unless the LTB orders or directs otherwise.

  Evidence should be submitted using the Tribunals Ontario Portal. Read the Practice Direction on Evidence for more information about how and when to submit evidence if you have an upcoming hearing. You must also serve the other parties with a copy of all the evidence you uploaded into the portal unless the other parties have consented in writing to accept service through the portal. The parties can use the Consent to Disclosure through Tribunals Ontario Portal form for such an agreement.

  At the hearing, the LTB member (adjudicator) will decide what documents will be accepted as evidence.

  Timelines for next steps
  On average, L1 and L9 applications are being scheduled within five months. A Notice of Hearing will be sent to the parties before the scheduled hearing date. This is an improvement from earlier this year, when the average was 8-10 months to get to a hearing.

  On average, other new and adjourned matters are currently being scheduled within seven to eight months from when they were received or adjourned.


          """)

            mediation = st.radio("Do you want to know about the Mediation and the Hearing Information?", ('Yes', 'No'))
            if mediation == 'Yes':
                st.write("""
Preparing for your hearing day
You might want to get legal advice before your hearing.

The LTB has rules and practice directions that apply to its processes, and interpretation guidelines that explain how the LTB might decide specific issues in an application. You can read them in Rules, Practice Directions and Guidelines. The Practice Direction on Evidence contains information about how and when to submit evidence if you have an upcoming hearing.

You might also want to read decisions of the LTB.

Mediation
During mediation, a neutral person talks with you and the other party to see if both sides can come up with a solution to the problem.

Parties using Tribunals Ontario Portal can use the Online Dispute Resolution (ODR) tool to negotiate, among themselves, a settlement of the issues in dispute.

If the parties reach an agreement, they can ask the LTB for assistance and a Dispute Resolution Officer (DRO) will be assigned to the matter. This is also the case if the parties reach an impasse. A DRO can help the parties determine if they wish to engage with mediation as the next step in the process, or if the matter will proceed to a hearing instead.

ODR is not mediation, and it’s not adjudication. It’s the first step in determining which of these two avenues is most appropriate based on the position of the parties. It is a negotiation between parties in a safe space which is monitored for abuse and which provides tips on negotiation. It’s also the space where people can request the assistance of a DRO.

ODR can result in early resolution of an application through a payment plan, consent order or mediated agreement with the assistance of a Dispute Resolution Officer.

ODR may also result in the applicant wishing to withdraw their application, in which case a withdrawal request can be submitted through the portal.

Some other benefits of ODR include easier access to a DRO, having all file information in one easily accessible place, and accessing documents/evidence quickly and easily.

Types of hearings
There are two types of hearings at the LTB. The LTB decides what type of hearing to hold.

During a telephone or video hearing, the parties file any documents or materials before the hearing. During the hearing, they give evidence and explain their position by telephone or by video. The LTB has information available for participating in a Zoom hearing and for participating in a telephone hearing.
For a written hearing, the parties file a written explanation of their position and supporting documents. The member considers the explanation and reviews the documents filed to make their decision. Written hearings are most commonly used to decide Applications for an Above Guideline Increase and Applications to Vary the Amount of a Rent Reduction.
Who goes to the hearing
Applicant(s): The person who filed the application (i.e. the landlord or the tenant) or a person who is appearing in their place.

Respondent(s): The other person named in the application (i.e. the landlord or the tenant) or a person who is appearing in their place.

The applicant and the respondent are called the parties to the application.

Witnesses: Witnesses are people who can give evidence (information) that is relevant to the dispute. Witnesses will usually be asked to give information rather than offer their opinion. For example, a witness may describe something they saw or heard.

You need to make sure that your witnesses participate at the right time, day and videoconference. If you are worried that a witness might not show up, you can complete a Request for Summons. A member will review the request and decide if a summons should be issued. If the member thinks the person will not be able to provide relevant evidence, the request will be refused.

If a witness needs to take time off work to go to the hearing, they might ask you for a summons to show to their employer.

If a witness is summoned to a hearing, you will need to personally deliver (“serve”) the summons to them and pay them witness fees. For more information, read the brochure Requesting a Summons.

Representatives: You do not need to have a legal representative, but if you do, the representative will also attend the hearing. Your representative will speak on your behalf at the hearing, question witnesses, submit evidence and make written submissions.

A representative may be a lawyer or paralegal you hire to act on your behalf. You may also choose a person who is not a lawyer or paralegal to represent you. This could be a friend or relative or someone who is allowed by the Law Society to provide legal services without a licence. If you cannot attend your hearing and your representative is not a lawyer or paralegal, you will need to give your representative permission in writing to act on your behalf. For more information, see the Practice Direction on Representation before the LTB.

If you don’t go the hearing and no one attends in your place, the member can dismiss the application or decide it without you.

Support Persons: A support person, such as a family member or friend, can attend a hearing or mediation with you. A support person is not considered a representative as long as they don’t speak for you in the hearing or mediation.

Litigation Guardians: Sometimes an application will involve a party without “legal capacity” either because the person does not have the mental capacity to make decisions about the issues in the case or, in certain cases, because the person is under 18. That person can participate in the hearing through a litigation guardian. For more information, see the Practice Direction on Litigation Guardians.

What happens during the hearing:
The notice of hearing gives you the date and time of your hearing. A number of files will have the same start time in what is called a “hearing block”.

Be sure to log onto your video link or call at least 15 minutes prior to your hearing. If you do not sign-in and you are not in the hearing room when your case is called, your hearing may go ahead without you.

Wait in the virtual video conference hearing room unless the member has said you can leave and come back at a certain time.

You may have to wait for much of the day, so make arrangements for child care or to be absent from work or school. If it’s not possible to finish your hearing that day you’ll be given a new hearing date.

During the hearing, you and the other party will have a chance to question witnesses, introduce relevant documents as evidence and make arguments about the facts and the law.

The member controls the hearing. Because the member is neutral, he or she cannot provide legal advice or tell you how to present your case. It is up to you to present evidence that supports your position. The member may ask questions during the hearing.

Everyone at the hearing is expected to be courteous and respectful. See Rules of Procedure A7.

When the hearing is over, the member might tell you their decision right away or they might “reserve” the decision, which means they will take more time to consider your evidence and submissions. In either case, you will receive the decision in writing explaining the result. This decision is called an order.

Resolving Matters Without a Hearing
In some cases, an application may be resolved without a hearing. Parties can resolve matters that are not complex using the forms below to help the LTB understand if there have been changes after an application has been submitted. The applicant can use the Request for a Discontinuance Order Without a Hearing Form and/or a Request to Withdrawal an Application form to tell the LTB that:

The applicant wants to withdraw the application
An order can be issued because the application is discontinued; the tenant has paid in full all arrears and the application fee
An order can be issued because the parties have agreed to a payment agreement
Applicants can submit the form using the Tribunals Ontario Portal. If you are unable to file on the portal, you can email the form to LTB@Ontario.ca. Please include your file number and the title of your request in the subject line.
        """)
            else:
                st.write("Thank you for contacting LTB, hope you already know informations about Mediation and hearing!")
                order = st.radio("Do you want to know more about the Orders that LTB issues?", ('Yes', 'No'))
                if order == 'Yes':
                    st.write("""
The member who hears the application (or reviews the documents filed, in the case of a written hearing) will make the final order.

The order tells the parties, in writing, what the member has decided and may describe terms or conditions that a party must follow. For example, the member could order that the tenancy is terminated, or that repairs be made to the rental unit.

Usually, the LTB will mail a copy of the order to all of the parties named in the application and their legal representative.

You can visit Navigate Tribunals Ontario to learn more about what you can do after receiving an order from the LTB.

About reasons
An order may include written reasons. The reasons explain how the member analysed the evidence to arrive at their decision.

You can ask for reasons to be included in the order:

while you are at the hearing, or
in writing, within 30 days of the date on the order.
Types of orders
There are four common orders:

an interim order
an ex parte order
a hearing order
a consent order
Interim order
An interim order is an order that tells a party to do something, or decides part of the application, before the final order is issued.

Ex-parte order
An ex parte order is made when the member reviews the documents filed by the applicant and makes a decision, without notifying the respondent. Only two application types can be decided ex parte:

An application filed by the landlord to end the tenancy because the tenant agreed to move out on a specific date or gave notice to move out on a specific date (L3 application)
An application filed by the landlord to end the tenancy because the tenant did not meet the terms of a mediated agreement or order (L4 application).
Hearing order
A member issues a hearing order after a hearing.

Consent order
The LTB can issue a consent order when:

a tenant and a landlord have settled their dispute through an agreement and ask for the terms of the settlement to be put into a consent order. The terms of a consent order (e.g. a tenant agrees to move out of the apartment on a specific date) can be enforced by the court.
a landlord files an application to evict a tenant for non-payment of rent (Form L1) or for arrears of rent (Form L9) if:
the landlord and tenant work out a payment plan, and
a copy of the written payment plan is filed with the LTB before the hearing.
In this case, no hearing is held and the consent order is based on the payment plan that landlord and the tenant have agreed to.

Voiding an LTB order
An eviction order issued by the LTB for non-payment of rent usually allows the tenant to void the order by paying all the money they owe by the deadline in the order.

If the tenant pays the landlord the money they owe before the deadline, the tenant can file a Tenant's Affidavit and Motion to Void an Eviction Order for Arrears of Rent. A member will decide if the eviction order is void without holding a hearing.

If the tenant pays all the money owed after the deadline, the tenant can file a Tenant's Affidavit and Motion to Void an Eviction Order for Arrears of Rent. In this case, a member will hold a hearing to decide if the eviction order is void.

Enforcement of an LTB order
Most of the terms and conditions of LTB orders can be enforced through the courts. For example, if the tenant does not move out of the rental unit by the date in the eviction order, the order can be filed with the Court Enforcement Office (also known as the Sheriff’s office) to be enforced. An order for payment of money can be filed with the Small Claims Court for enforcement.

Timelines for receiving an order
As of June 2023, on average, orders are being sent within 60 days of a hearing.
        """)
                else:
                    st.write("Thank you for contacting LTB, hope you already know informations about Orders that LTB issues!")
                    challenge = st.radio("Do you want to know more about Challenging or Correcting the order?", ('Yes', 'No'))
                    if challenge == 'Yes':
                        st.write("""
Once the LTB issues an order, it is final. The LTB will not change the order because a party does not like the decision or because a party believes that a different decision should have been made. For more information, read the brochure: Amendments, Reviews and Appeals.

Amending an order
If you notice a clerical mistake in the order, such as a spelling error in someone’s name or a calculation error, you have 30 days to file a Request to Amend an Order to ask the member to correct the mistake. In your request, tell the member what the mistake is and why it should be corrected. There is no fee to make this request. For details, see Interpretation Guideline 15: Amending an Order.

Asking the LTB to review an order
You can ask the LTB to review an order on the grounds that a “serious error” has been made in the order. A serious error can be:

LTB made an order it did not have the power to make
The application was not processed fairly and this interfered with the rights of one of the parties
The hearing was not conducted fairly and this interfered with the rights of one of the parties
There was no evidence to support the LTB’s conclusions
New evidence is available which was not available at the time of the hearing
A party was not reasonably able to participate in the LTB’s proceeding.
You need to file a Request to Review an Order within 30 days of the date of the order. There is a $58.00 fee.

You must explain in detail the error you believe the member made, and specify the changes you think should be made to the order.

If you want the order stayed (or put “on-hold’) until a decision can be made, you must ask for the stay, and give reasons why.

Review orders can:

deny the request for review
affirm the hearing order
order changes to the hearing order
For details, see Interpretation Guideline 8: Review of an Order.

Appealing an order
Any person affected by an LTB order can appeal the order to the Divisional Court. This court is the branch of the Superior Court of Justice that hears appeals from the decisions of boards and tribunals in Ontario.

An appeal of an order may only be made on a question of law. In general, this means a person must believe LTB’s order was not a reasonable interpretation or application of the law.

An appeal to Divisional Court must be filed within 30 days of the date the order was issued.

When an appeal of an order is filed with the Divisional Court, the order is automatically stayed, and cannot be enforced unless the Divisional Court or the LTB lifts (or removes) the stay at the request of a party.

The procedures for filing an appeal to the Divisional Court are set by the court, not by the LTB. You can contact the courthouse for more information about the appeal process. You can also download the Guide to Appeals in Divisional Court from the Divisional Court's website. You may want to get legal help.

Setting aside an ex parte order
The following applications may result in an ex parte order:

an application filed by the landlord to end the tenancy because the tenant agreed to move out on a specific date or gave notice to move out on a specific date (L3 application),
an application filed by the landlord to end the tenancy because the tenant did not meet the terms of a mediated agreement or order (L4 application).
If an ex parte order is issued, you can ask the LTB to set aside (or cancel) the order. A request to set aside the order is called a motion, which can be filed with the LTB by completing a Motion to Set Aside an Ex Parte Order. The motion must be filed with the LTB within 10 days from the date the order is issued. A hearing will be held and the member will make a decision.

When a motion to set aside an order is filed by the deadline, the order is automatically stayed (or put on hold) and it cannot be enforced until a final decision has been made.

For more information, see Motion to Set Aside an Ex Parte Order.

Re-opening an application
Only applications that were resolved by a consent order under section 206 of the RTA or an LTB mediated agreement can be re-opened.

There are two common situations when you can make the request:

The landlord or tenant did not meet one of the terms in the LTB mediated agreement
The landlord or tenant misled you or made you sign the mediated agreement
To ask to re-open an application, fill out the Request to Re-open an Application.
        """)
                        st.write("Thanks for contacting LTB, Hope you had fun!")

if __name__ == "__main__":
    main()


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 31)

In [14]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


In [ ]:
streamlit run app.py